# 3.3.1 生成数据集

In [1]:
import torch
import numpy as np

num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

# 3.3.2 读取数据
pytorch提供data包读取数据，每次迭代中，随机读取包含10个数据样本的小批量

In [2]:
import torch.utils.data as Data

batch_size = 10
# 将训练数据的特征和标签组合
dataset = Data.TensorDataset(features, labels)
# 随机读取小批量
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)
print("这里的data_iter可以让我们读取并打印第一个小批量数据样本")

for X, y in data_iter:
    print(X, y)
    break

这里的data_iter可以让我们读取并打印第一个小批量数据样本
tensor([[ 0.8948, -0.7860],
        [ 0.5317,  0.7145],
        [-0.4846,  0.7632],
        [-0.8772,  1.8602],
        [ 0.1564,  2.1424],
        [-1.2207,  1.5772],
        [-1.8153,  0.5119],
        [ 0.4763,  0.8826],
        [-0.8467, -1.0120],
        [ 1.4520,  0.0541]]) tensor([ 8.6572,  2.8513,  0.6372, -3.8756, -2.7882, -3.5887, -1.1709,  2.1485,
         5.9524,  6.9147])


# 3.3.3 定义模型
pytorch提供了大量预定义的层，使我们只需关注使用那些层来构造模型。需要导入torch.nn模块，该模块定义了大量神经网络的层，而nn就是利用autograd来定义模型的。nn的核心数据结构是Module，它既可以表示神经网络的某个层，也可以表示一个包含很多层的神经网络。实际使用中，最常用的做法就是继承nn.Module。  
一个nn.Module的实例应该包含一些层以及返回输出的前向传播（forward）方法。

In [3]:
# 使用nn.Module实现一个线性回归模型
import torch.nn as nn

class LinearNet(nn.Module):
    def __init__(self, n_feature):
        super(LinearNet, self).__init__()
        self.linear = nn.Linear(n_feature, 1)
    # forward 定义前向传播
    def forward(self, x):
        y = self.linear(x)
        return y
    
net = LinearNet(num_inputs)
print(net)

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


另外还可以使用 nn.Sequential 更加方便搭建网络，Sequential是一个有序的容器，网络层将按照在传入Sequential的顺序依次被添加到计算图中

In [4]:
# 写法一
net = nn.Sequential(
    nn.Linear(num_inputs, 1)
    # 此处还可以传入其他层
)

# 写法二
net = nn.Sequential()
net.add_module('linear', nn.Linear(num_inputs, 1))
# net.add_module ......

# 写法三
from collections import OrderedDict

net = nn.Sequential(OrderedDict([
          ('linear', nn.Linear(num_inputs, 1))
          # ......
]))

print(net)
print(net[0])

Sequential(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)
Linear(in_features=2, out_features=1, bias=True)


In [5]:
print("通过net.parameters()来查看模型所有可学习的参数，此函数返回一个生成器")

for param in net.parameters():
    print(param)

通过net.parameters()来查看模型所有可学习的参数，此函数返回一个生成器
Parameter containing:
tensor([[-0.2233, -0.5545]], requires_grad=True)
Parameter containing:
tensor([-0.2917], requires_grad=True)


## 注意：torch.nn仅支持输入一个batch的样本不支持单个样本输入，要输入单个样本，可使用input.unsqueeze(0)来添加一维

# 3.3.4 初始化模型参数
pytorch在 init 模块中提供了多种参数初始化方法

In [6]:
from torch.nn import init

# import torch.nn as nn

# class LinearNet(nn.Module):
#     def __init__(self, n_feature):
#         super(LinearNet, self).__init__()
#         self.linear = nn.Linear(n_feature, 1)
#     # forward 定义前向传播
#     def forward(self, x):
#         y = self.linear(x)
#         return y
    
# net = LinearNet(num_inputs)

init.normal_(net[0].weight, mean=0, std=0.01) # 将权重的每个参数初始化为均值为 0，标准差为 0.01的正态分布，偏差初始化为 0
init.constant_(net[0].bias, val=0) # 也可以直接修改bias的data：net[0].bias.data.fill_(0)

Parameter containing:
tensor([0.], requires_grad=True)

### 注：如果这里的 net是用自定义继承nn.Module类的方法创建的，上面的代码会报错，net[0].weight应改为 net.linear.weight，bias 也是，net[0]这样根据下标访问子模块的写法只有当 net是一个 ModuleList或者 Sequential实例才可以

# 3.3.5 定义损失函数
nn模块中定义各种损失函数，可看做是特殊的层，pytorch也将这些损失函数实现为nn.Module的子类

In [7]:
loss = nn.MSELoss() # 使用均方误差作为模型的损失函数

# 3.3.6 定义优化算法
torch.optim模块提供很多常用的优化算法如 SGD、Adam和RMSProp

In [8]:
print("下面创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量 SGD为优化算法")
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

下面创建一个用于优化net所有参数的优化器实例，并指定学习率为0.03的小批量 SGD为优化算法
SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


In [9]:
print("可以为不同子网络设置不同的学习率，在finetune时经常用到")
# optimizer = optim.SGD([
#     # 如果对某个参数不指定学习率，就使用最外层的默认学习率
#     {'params': net.subnet1.parameters()}, # lr=0.03
#     {'params': net.subnet2.parameters(), 'lr':0.01}
# ], lr=0.03)

可以为不同子网络设置不同的学习率，在finetune时经常用到


如果不想让学习率固定为一个常数，那么如何调整？一种是修改 optimizer.param_groups中对应的学习率，另一种是——新建优化器，optimizer比较轻量级，开销很小，可以构建新的optimizer。但是后者对于使用动量的优化器（如Adam），会丢失动量等状态信息，可能造成损失函数的收敛出现震荡

In [10]:
# 调整学习率
for param_group in optimizer.param_groups:
    param_group['lr'] *= 0.1

# 3.3.7 训练模型
通过调用 optim实例的 step函数迭代模型参数，按照小批量随机梯度下降的定义，在 step函数中指明批量大小，从而对批量中样本梯度求平均。

In [15]:
num_epochs = 3

for epoch in range(1, num_epochs+1):
    for X, y in data_iter:
        output = net(X)
        l = loss(output, y.view(-1, 1))
        optimizer.zero_grad() # 梯度清零，等价于net.zero_grad()
        #net.zero_grad()
        l.backward()
        optimizer.step()
    print('epoch %d, loss: %f' %(epoch, l.item()))

epoch 1, loss: 0.004242
epoch 2, loss: 0.002600
epoch 3, loss: 0.000624


In [16]:
print("比较学到的模型参数和真实参数")
dense = net[0]
print(true_w, dense.weight)
print(true_b, dense.bias)

比较学到的模型参数和真实参数
[2, -3.4] Parameter containing:
tensor([[ 1.9991, -3.3830]], requires_grad=True)
4.2 Parameter containing:
tensor([4.1825], requires_grad=True)
